# Объединение результатов работы Image Captioning + ASR

В рамках препроцессинга, когда испытывали сложности по времени обработки и железу. Решили распаралелить процесс сборки данных. 
Логика следующая:
- есть результаты работы ImageCaptioning чанками
- есть результаты работы ASR чанками
- построили обработку так, чтобы записи из исходного датасета обрабатывались одинаково параллельно. +- одна и та же запись попадает на предикт в две модели
- конкатенируем чанки в 1 Датафрейм
- присоединяем к датафрейму с ImageCaptioning результат ASR
- сохраняем в объединенный csv


In [1]:
import pandas as pd
import os

In [2]:
speech_path = './/speech/'
captioning_path = './/captioning/'

output_speech_path = './/speech_1.csv'
output_captioning_path = './/union_1.csv'

In [3]:
concated_speech_frame = None
concated_captionin_frame = None

In [4]:
for dirname, _, filenames in os.walk(speech_path):
    i = 0
    for filename in filenames:
        if '.csv' in filename:
            if i == 0:
                df_path = str(os.path.join(dirname, filename))
                concated_speech_frame = pd.read_csv(df_path)
                i+=1
            else:
                df_path = str(os.path.join(dirname, filename))
                temp_df = pd.read_csv(df_path)
                concated_speech_frame = pd.concat([concated_speech_frame, temp_df])
                
display(concated_speech_frame.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1591 entries, 0 to 98
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   link    1591 non-null   object
 1   tags    1382 non-null   object
 2   text    1130 non-null   object
dtypes: object(3)
memory usage: 49.7+ KB


None

In [5]:
for dirname, _, filenames in os.walk(captioning_path):
    i = 0
    for filename in filenames:
        if '.csv' in filename and filename != 'test_0.csv':
            if i == 0:
                df_path = str(os.path.join(dirname, filename))
                concated_captionin_frame = pd.read_csv(df_path)
                i+=1
            else:
                df_path = str(os.path.join(dirname, filename))
                temp_df = pd.read_csv(df_path)
                concated_captionin_frame = pd.concat([concated_captionin_frame, temp_df])

display(concated_captionin_frame.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1498 entries, 0 to 497
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   link                  1498 non-null   object
 1   tags                  1302 non-null   object
 2   description_ru        1498 non-null   object
 3   short_description_ru  1498 non-null   object
 4   description_en        1498 non-null   object
 5   short_description_en  1498 non-null   object
dtypes: object(6)
memory usage: 81.9+ KB


None

In [6]:
def find_speech_by_link(row):
    link = row['link']
    series = concated_speech_frame[concated_speech_frame['link'] == link]['text']
    return series.values[0] if series.size > 0 else None

In [7]:
concated_captionin_frame['text'] = concated_captionin_frame.apply(find_speech_by_link, axis=1)

In [8]:
concated_captionin_frame.to_csv(output_captioning_path, index=False)
#concated_speech_frame.to_csv(output_speech_path, index=False)

In [9]:
concated_captionin_frame['text'].head()

0     Вы видели, как на самом деле выглядит Чупакаб...
1     Редактор субтитров А.Синецкая Корректор А.Его...
2                                                  NaN
3     Пропаганда во все времена в любой стране испо...
4                                                  NaN
Name: text, dtype: object

In [10]:
pd.set_option('display.expand_frame_repr', False)

In [11]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [13]:
from pandas import option_context

with option_context('display.max_colwidth', 400):
    display(concated_captionin_frame.head())


,link,tags,description_ru,short_description_ru,description_en,short_description_en,text
0,https://cdn-st.rutubelist.ru/media/00/4b/0bbb24d04c3289605e329c0d3ee8/fhd.mp4,#путешествия #кудапоехать #туризм #море #горы #советыдляпутешествий #кэмпинг #глэмпинг #отель #вдохновение #влог,"Человек в галстуке и шляпу. Человек с бородой и шляпой, держащей овцу. Человек со шляпой и собакой в ​​клетке. Мужчина в шляпе и галстук с изображением женщины. Человек со шляпой и борода, держащий нож. Человек в шляпе и галстуке держит корзину. Человек со шляпой и шляпой на голове. Человек со шляпой и солнцезащитными очками с мобильным телефоном. Человек в шляпе и галстук. Человек в шляпе и г...","Человек в шляпе и галстуке, позирующем для картинки.",A man in a tie and a hat. A man with a beard and a hat holding a sheep. A man with a hat and a dog in a cage. A man in a hat and tie with a picture of a woman. A man with a hat and a beard holding a knife. A man in a hat and tie is holding a basket. A man with a hat and a hat on his head. A man with a hat and sunglasses holding a cell phone. A man wearing a hat and a tie. A man in a hat and ti...,A man in a hat and tie posing for a picture.,"Вы видели, как на самом деле выглядит Чупакабра? 1975 год. В пуэрто-риканской деревне некто нападает на животных и высасывает у них всю кровь. Изначально в этом обвиняют некий сатанистский культ, но позже ситуация повторяется по всему Пуэрто-Рико. Овец, кос и других обескровленных животных находятся с тремя точками на шее. Жуткое создание, высасывающее у них все соки, так и окрестили Чупакабр..."
1,https://cdn-st.rutubelist.ru/media/00/4b/0e058b624c3fa0e44a101b7d94f9/fhd.mp4,#спорт #наспорте #растяжка #фитнес #мотивация,"Мужчина в синей рубашке и синие брюки играет в игру в Фрисби. Мужчина в синей рубашке катается на коньках на снежном склоне. Мужчина в синей рубашке и синих штанах играет в игру в Фрисби. Человек, катаясь на скейтборде по снежному склону. Человек в синей рубашке и черных брюках играет в игру в Фрисби. Мужчина в синей рубашке и синих штанах стоит в комнате. Женщина в синей рубашке и черных брюк...",Мужчина в синей рубашке и синие брюки играет в игру в Фрисби.,A man in blue shirt and blue pants is playing a game of frisbee. A man in a blue shirt is skating on a snow covered slope. A man in a blue shirt and blue pants is playing a game of frisbee. A man riding a skateboard down a snow covered slope. A man in a blue shirt and black pants is playing a game of frisbee. A man in a blue shirt and blue pants is standing in a room. A woman in a blue shirt a...,A man in blue shirt and blue pants is playing a game of frisbee.,Редактор субтитров А.Синецкая Корректор А.Егорова
2,https://cdn-st.rutubelist.ru/media/00/4b/2e2a59f74a83b2909436bb9da1ac/fhd.mp4,#авто #auto #vehicle #тачки #bmw,Большая картина часов на столе. Цветочная композиция в вазе на столе. Большая синяя и белая скульптура часов. Размытое изображение цветка в воздухе. Одеяло сидит на столе в комнате.,Размытое изображение цветка в воздухе.,A large painting of a clock on a table. A flower arrangement in a vase on a table. A large blue and white sculpture of a clock. A blurry image of a flower in the air. A blanket is sitting on a table in a room.,A blurry image of a flower in the air.,NaN
3,https://cdn-st.rutubelist.ru/media/00/4b/99f3ec48433488d52c62c43cec5a/fhd.mp4,"#егэ2023 , #огэ2023 , #наука , #егэ , #огэ , #научпоп","Картина человека в бейсбольной форме. Знак, который гласит «Не переставай пить» на машине. Фотография группы людей в комнате. Изображение человека, носящего шляпу и галстук. Картина человека в униформе с флагом. Фотография девушки с мобильным телефоном. Фотография женщины с изображением мужчины. Человек в костюме разговаривает по мобильному телефону. Фотография человека, стоящего перед знаком....",Человек держит игровой контроллер Nintendo Wii.,"A picture of a man in a baseball uniform. A sign that reads ""Don't Stop Drinking"" on a car. A picture of a group of people in a room. A